In [1]:
import numpy as np
import pandas as pd

In [2]:
first_df1 = pd.read_csv("modeling data/1_players_info_added_Clean_data.csv")
df1_nulls = first_df1.isnull().sum()
print(df1_nulls)
print(len(first_df1))

Unnamed: 0           0
Location             0
Date                 0
Round                0
Winner               0
Loser                0
WRank               60
LRank              206
W_hand               0
W_ht               957
W_nationality      858
W_age              859
L_hand               0
L_ht              1102
L_nationality      791
L_age              796
L_wt             11818
W_wt             11801
dtype: int64
19922


In [3]:
df_drop_nulls = first_df1.copy()
df_drop_nulls = df_drop_nulls.dropna()
len(df_drop_nulls)

7399

In [4]:
df1_dropped_wt = first_df1.copy()
df1_dropped_wt.drop(columns=["Unnamed: 0",'L_wt', 'W_wt'], inplace=True)
df1_no_nulls = df1_dropped_wt.dropna()
len(df1_no_nulls)

17917

In [5]:
def addingWinnerColumn(df):
    df_winner = df.copy()
    df_loser = df.copy()

    df_winner['win'] = 1
    df_loser['win'] = 0

    # Renaming columns to 'player' and 'opponent' for winners
    df_winner.rename(columns={
        'Winner': 'player',
        'Loser': 'opponent',
        'WRank': 'player_rank',
        'LRank': 'opponent_rank',
        'W_hand': 'player_hand',
        'L_hand': 'opponent_hand',
        'W_ht': 'player_ht', 
        'W_nationality':'player_nationality', 
        'W_age':'player_age', 
        'L_ht':'opponent_ht',
        'L_nationality':'opponent_nationality', 
        'L_age':'opponent_age'
    }, inplace=True)

    # Renaming columns to 'player' and 'opponent' for losers
    df_loser.rename(columns={
        'Winner': 'opponent',
        'Loser': 'player',
        'WRank': 'opponent_rank',
        'LRank': 'player_rank',
        'W_hand': 'opponent_hand',
        'L_hand': 'player_hand',
        'W_ht':'opponent_ht', 
        'W_nationality':'opponent_nationality', 
        'W_age':'opponent_age', 
        'L_ht':'player_ht',
        'L_nationality':'player_nationality',
        'L_age':'player_age'
    }, inplace=True)
    df_transformed = pd.concat([df_winner, df_loser], ignore_index=True)
    df_transformed = df_transformed.sample(frac=1).reset_index(drop=True)
    return df_transformed

In [6]:
def fixPlayerRank(df):
    df_transformed = df.copy()
    player_peak_ranking = df_transformed.groupby('player')['player_rank'].min().reset_index()
    opponent_peak_ranking = df_transformed.groupby('opponent')['opponent_rank'].min().reset_index()
    player_peak_ranking

    player_updated = pd.merge(df_transformed, player_peak_ranking[['player', 'player_rank']], on='player', how='left')
    opponent_updated = pd.merge(player_updated, opponent_peak_ranking[['opponent', 'opponent_rank']], on='opponent', how='left')
    df1 = opponent_updated.copy()
    df1.drop(columns=['player_rank_x', 'opponent_rank_x'], inplace=True)
    df1 = df1.rename(columns={  'player_rank_y': 'player_rank',
                                'opponent_rank_y': 'opponent_rank'})
    return df1

In [7]:
countryList = [
    ["AUS", "Australia", "Canberra", "Sydney", "Melbourne", "Brisbane", "Perth", "Adelaide"],
    ["IND", "India", "New Delhi", "Mumbai", "Kolkata", "Chennai", "Bangalore", "Hyderabad", "Pune"],
    ["QAT", "Qatar", "Doha"],
    ["NZL", "New Zealand", "Wellington", "Auckland", "Christchurch"],
    ["FRA", "France", "Paris", "Marseille", "Lyon", "Toulouse", "Nice", "Nantes", "Strasbourg", "Montpellier", "Bordeaux", "Lille","Roland Garros","Metz"],
    ["ECU", "Ecuador", "Quito", "Guayaquil"],
    ["BGR", "Bulgaria", "Sofia", "Plovdiv", "Varna"],
    ["ARG", "Argentina", "Buenos Aires", "Cordoba", "Rosario", "Mendoza"],
    ["USA", "US", "United States", "New York", "Los Angeles", "Chicago", "Houston", "Phoenix", "Philadelphia", "San Antonio", "San Diego", "Dallas", "Salem", "Cincinnati","Newport",'Atlanta','Washington','Memphis',"Delray Beach","Miami"],
    ["BRA", "Brazil", "Brasília", "São Paulo", "Sao Paulo","Rio de Janeiro","Rio", "Salvador", "Fortaleza", "Belo Horizonte"],
    ["MEX", "Mexico", "Mexico City", "Guadalajara", "Monterrey", "Puebla", "Toluca", "Tijuana","Acapulco","Los Cabos"],
    ["ARE", "United Arab Emirates", "Abu Dhabi", "Dubai", "Sharjah"],
    ["MAR", "Morocco", "Rabat", "Casablanca", "Fez", "Marrakech"],
    ["MCO", "Monaco", "Monte Carlo"],
    ["ESP", "Spain", "Madrid", "Barcelona", "Valencia", "Seville", "Zaragoza"],
    ["HUN", "Hungary", "Budapest"],
    ["PRT", "Portugal", "Lisbon", "Porto", "Amadora", "Braga","Estoril"],
    ["TUR", "Turkey", "Ankara", "Istanbul", "Izmir", "Bursa", "Antalya"],
    ["DEU", "Germany", "Berlin", "Hamburg", "Munich", "Cologne", "Frankfurt", "Stuttgart", "Düsseldorf","Dusseldorf", "Dortmund", "Essen", "Leipzig","Halle"],
    ["ITA", "Italy", "Rome", "Milan", "Naples", "Turin", "Palermo", "Genoa", "Bologna", "Florence","nitto","Sardinia"],
    ["CHE", "Switzerland", "Zurich", "Geneva", "Basel", "Lausanne","Gstaad"],
    ["NLD", "Netherlands", "Amsterdam", "Rotterdam", "The Hague", "Utrecht","s Hertogenbosch"],
    ["GBR", "United Kingdom", "London", "Birmingham", "Glasgow", "Liverpool", "Leeds", "Sheffield", "Edinburgh", "Bristol", "Manchester","Eastbourne","wimbledon"],
    ["SWE", "Sweden", "Stockholm", "Gothenburg", "Malmö","Malmo", "Uppsala",'bastad'],
    ["HRV", "Croatia", "Zagreb", "Split", "Rijeka",'Umag'],
    ["CHN", "China", "Beijing", "Shanghai", "Chongqing", "Tianjin", "Guangzhou", "Shenzhen", "Chengdu","Zhuhai"],
    ["BEL", "Belgium", "Brussels", "Antwerp", "Ghent", "Charleroi", "Liège"],
    ["RUS", "Russia", "Moscow", "Saint Petersburg","st petersburg","Petersburg" "Novosibirsk", "Yekaterinburg", "Nizhny Novgorod"],
    ["CHL", "Chile", "Santiago"],
    ["CAN", "Canada", "Ottawa", "Toronto", "Montreal", "Calgary", "Edmonton", "Vancouver", "Winnipeg", "Quebec City", "Hamilton", "Kitchener"],
    ["KAZ", "Kazakhstan", "Nur Sultan", "Almaty"],
    ["AUT", "Austria", "Vienna","Kitzbuhel"],
    ["JPN", "Japan", "Tokyo", "Yokohama", "Osaka", "Nagoya", "Sapporo", "Fukuoka", "Kobe", "Kyoto"],
    ["Unknown", "Davis","Club",'Cup', 'Tour Finals',"NextGen"]
]

In [8]:
import re
def fixCountryName(df):
    newCountryName = df.copy()
    newCountryName["Location"] = newCountryName["Location"].str.replace('_'," ")
    newCountryName["Location"] = newCountryName["Location"].str.replace('-'," ")

    for country in countryList:
        for i in range(len(country)):
            newCountryName["Location"] = newCountryName["Location"].str.replace('.*'+country[i]+'.*', country[0], flags=re.IGNORECASE, regex=True)
    return newCountryName

In [9]:
def isHomeCountry(df):
    df1_home_country = df.copy()
    df1_home_country['is_player_home_country'] = np.equal(df1_home_country['Location'], df1_home_country['player_nationality'])
    df1_home_country['is_opponent_home_country'] = np.equal(df1_home_country['Location'], df1_home_country['opponent_nationality'])
    df1_home_country.drop(columns=["Location",'player_nationality', 'opponent_nationality'], inplace=True)
    return df1_home_country

In [10]:
def split_date(df):
    df1 = df.copy()
    df1['Date'] = pd.to_datetime(df1['Date'])
    df1['year'] = df1['Date'].dt.year
    df1['month'] = df1['Date'].dt.month
    df1['day'] = df1['Date'].dt.day
    df1['dayofweek'] = df1['Date'].dt.dayofweek
    df1.drop(columns=["Date"], inplace=True)
    return df1

In [11]:
def startToHome(df):
    df_target = df.copy()
    df_target_win = addingWinnerColumn(df_target)
    df_target_player = fixPlayerRank(df_target_win)
    df_target_country = fixCountryName(df_target_player)
    df_target_home = isHomeCountry(df_target_country)
    df_target_dates = split_date(df_target_home)
    return df_target_dates

In [12]:
# df_home = pd.read_csv("modeling data/df1_with_home.csv")
df_home = startToHome(df1_no_nulls)
df_drop_nulls = startToHome(df_drop_nulls)

raw_df = startToHome(first_df1)
raw_df

,Unnamed: 0,Round,player,opponent,player_hand,player_ht,player_age,opponent_hand,opponent_ht,opponent_age,...,W_wt,win,player_rank,opponent_rank,is_player_home_country,is_opponent_home_country,year,month,day,dayofweek
0,1649,Semifinals,Querrey S.,Cilic M.,R,198.0,31.2,R,198.0,30.2,...,NaN,0,11.0,3.0,False,False,2017,7,14,4
1,4306,Round of 64,Fognini F.,Ymer M.,R,178.0,31.6,R,183.0,20.5,...,NaN,1,9.0,67.0,False,True,2018,7,19,3
2,6562,Round of 64,Struff J.L.,Albot R.,U,NaN,NaN,R,175.0,29.1,...,NaN,1,33.0,39.0,False,False,2019,5,30,3
3,7021,Round of 64,Fabbiano T.,Sonego L.,R,173.0,29.6,R,191.0,23.7,...,NaN,1,70.0,32.0,False,False,2019,7,24,2
4,7198,Round of 32,Maden Y.,Kicker N.,R,185.0,35.0,R,178.0,32.0,...,68.0,1,109.0,84.0,False,False,2017,1,1,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39839,2230,Round of 16,Mahut N.,Querrey S.,R,191.0,42.0,R,198.0,37.0,...,95.0,0,38.0,11.0,False,False,2016,1,1,4
39840,1950,Round of 64,Herbert P.,Shapovalov D.,R,188.0,28.3,L,185.0,20.3,...,NaN,0,36.0,11.0,False,True,2019,8,5,0
39841,4853,Round of 32,Cervantes I.,Olivo R.,R,183.0,35.0,R,180.0,32.0,...,73.0,0,125.0,79.0,False,False,2017,1,1,6
39842,6809,Round of 128,Kyrgios N.,Thompson J.,R,193.0,23.6,R,183.0,24.6,...,NaN,1,13.0,43.0,False,False,2019,7,2,1


In [13]:
df_outlier1 = df_home.copy()
def detect_outliers(df, column, threshold=4):
    """
    Detect outliers in a dataframe column using the IQR method.

    Returns:
        list: A list containing the indices of the outliers.
    """
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - threshold * IQR
    upper_bound = Q3 + threshold * IQR
    outliers = df[(df[column] < lower_bound) | (df[column] > upper_bound)].index.tolist()
    return outliers


numeric_columns = ['player_ht', 'player_age', 'opponent_ht', 'opponent_age', 'player_rank', 'opponent_rank']
outliers = {}
outliers_count={}
for column in numeric_columns:
    outliers[column] = detect_outliers(df_outlier1, column)
    outliers_count[column] = len(outliers[column])


# Print out indices of outliers for each column
for column, indices in outliers.items():
    print(f"Outliers in column '{column}': {indices}, Count: {outliers_count[column]}")


Outliers in column 'player_ht': [], Count: 0
Outliers in column 'player_age': [], Count: 0
Outliers in column 'opponent_ht': [], Count: 0
Outliers in column 'opponent_age': [], Count: 0
Outliers in column 'player_rank': [5, 16, 52, 61, 80, 131, 166, 193, 224, 231, 249, 275, 310, 332, 333, 359, 384, 403, 414, 467, 504, 573, 588, 593, 605, 624, 628, 652, 796, 805, 839, 840, 846, 854, 863, 892, 893, 903, 911, 928, 958, 964, 974, 994, 1006, 1012, 1018, 1036, 1077, 1129, 1168, 1198, 1217, 1313, 1333, 1369, 1386, 1397, 1485, 1495, 1528, 1535, 1570, 1576, 1583, 1595, 1616, 1643, 1714, 1775, 1783, 1881, 1886, 1904, 1922, 1997, 2010, 2011, 2014, 2129, 2179, 2202, 2208, 2221, 2301, 2303, 2307, 2318, 2332, 2344, 2352, 2363, 2434, 2446, 2451, 2460, 2462, 2593, 2637, 2651, 2678, 2697, 2741, 2754, 2861, 2887, 2894, 2910, 2926, 2986, 3005, 3023, 3027, 3046, 3071, 3076, 3091, 3125, 3159, 3184, 3201, 3206, 3208, 3213, 3231, 3249, 3262, 3281, 3296, 3349, 3391, 3434, 3454, 3485, 3542, 3556, 3598, 3606, 3

In [14]:
from scipy import stats

df_outlier2 = df_home.copy()

import numpy as np

z_scores = np.abs(stats.zscore(df_outlier2[numeric_columns]))
filtered_entries = (z_scores < 3).all(axis=1)
df_outlier2 = df_outlier2[filtered_entries]
df_outlier2

,Round,player,opponent,player_hand,player_ht,player_age,opponent_hand,opponent_ht,opponent_age,win,player_rank,opponent_rank,is_player_home_country,is_opponent_home_country,year,month,day,dayofweek
0,Round of 16,Johnson S.,Harris L.,R,188.0,30.8,R,193.0,23.6,0,25.0,72.0,False,False,2020,10,12,0
1,Round of 128,Melzer G.,Monteiro T.,L,188.0,34.0,L,183.0,24.7,0,68.0,74.0,False,False,2018,5,1,1
2,Round of 32,Benneteau J.,Fognini F.,R,185.0,43.0,R,178.0,31.6,0,55.0,9.0,False,False,2018,1,20,5
3,Round of 64,Querrey S.,Young D.,R,198.0,31.2,L,183.0,35.0,0,11.0,42.0,False,False,2017,3,12,6
4,Semifinals,Dzumhur D.,Basic M.,R,172.0,26.7,R,188.0,27.5,1,23.0,75.0,False,False,2017,10,21,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35829,Round of 64,Ramos-Vinolas A.,Cecchinato M.,L,188.0,36.0,R,185.0,26.2,1,17.0,16.0,False,False,2019,3,9,5
35830,Round of 32,Novak D.,Darcis S.,R,183.0,25.8,R,178.0,35.3,1,85.0,38.0,False,False,2019,7,15,0
35831,Round of 64,Daniel T.,Nadal R.,R,191.0,25.9,L,185.0,32.6,0,69.0,1.0,False,False,2017,9,1,4
35832,Round of 128,Kukushkin M.,Klizan M.,R,183.0,31.4,L,190.0,29.8,0,40.0,34.0,False,False,2019,5,27,0


In [15]:
#encoding the date using sin and cos to preserve the cyclical nature of
def encode_date(df):
    df_date_encoded = df.copy()
    df_date_encoded['month_sin'] = np.sin(2 * np.pi * df_date_encoded['month']/12)
    df_date_encoded['month_cos'] = np.cos(2 * np.pi * df_date_encoded['month']/12)
    df_date_encoded['dayofweek_sin'] = np.sin(2 * np.pi * df_date_encoded['dayofweek']/7)
    df_date_encoded['dayofweek_cos'] = np.cos(2 * np.pi * df_date_encoded['dayofweek']/7)
    df_date_encoded['day_sin'] = np.sin(2 * np.pi * df_date_encoded['day']/30.5)
    df_date_encoded['day_cos'] = np.cos(2 * np.pi * df_date_encoded['day']/30.5)
    df_date_encoded.drop(columns=["month","dayofweek", "day"], inplace=True)
    return df_date_encoded

In [16]:
#encoding rounds, no need to scale because the higher the round is, the more important it is
def encode_round(df):
    df_round_encoded = df.copy()
    round_mapping = {
        '1st Round Qualifying': 1,
        '2nd Round Qualifying': 2,
        '3rd Round Qualifying': 3,
        'Round Robin': 4,
        'Round of 128': 5,
        'Round of 64': 6,
        'Round of 32': 7,
        'Round of 16': 8,
        'Quarterfinals': 9,
        'Semifinals': 10,
        'The Final': 11
    }
    df_round_encoded['Round'] = df_round_encoded['Round'].map(round_mapping)
    return df_round_encoded

In [17]:
#binary encoding for columns in columnNames = ['is_player_home_country','is_opponent_home_country',"is_taller", "is_older", "rank_is_higher"]
def binary_column(df):
    df_binary = df.copy()
    df_binary['is_older'] = df_binary['player_age'] > df_binary['opponent_age']
    df_binary['is_taller'] = df_binary['player_ht'] > df_binary['opponent_ht']
    df_binary['rank_is_higher'] = df_binary['player_rank'] > df_binary['opponent_rank']

    df_binary.drop(columns=["player_age",'opponent_age', 'player_ht',"opponent_ht",'player_rank', 'opponent_rank'], inplace=True)
    return df_binary

In [18]:
#integer encoding for handedness
def int_encode_hand(df):
    df_hand_int_encoded = df.copy()
    hand_mapping = {
        'R': 1,
        'L': 2,
        'U': 0
    }
    df_hand_int_encoded['player_hand'] = df_hand_int_encoded['player_hand'].map(hand_mapping)
    df_hand_int_encoded['opponent_hand'] = df_hand_int_encoded['opponent_hand'].map(hand_mapping)
    return df_hand_int_encoded

In [19]:
#one-hot encoding for handedness
def hot_encode_hand(df):
    df_hand_1hot_encoded = df.copy()

    df_hand_1hot_encoded = pd.get_dummies(df_hand_1hot_encoded, columns=['player_hand'], prefix='player_hand')
    df_hand_1hot_encoded = pd.get_dummies(df_hand_1hot_encoded, columns=['opponent_hand'], prefix='opponent_hand')
    return df_hand_1hot_encoded

In [20]:
def encode_pipeline_L1(df):
    df1 = df.copy()
    df2 = encode_date(df1)
    df3 = encode_round(df2)
    df_int_hand = int_encode_hand(df3)
    df_hot_hand = hot_encode_hand(df3)
    return df_int_hand,df_hot_hand,binary_column(df_int_hand),binary_column(df_hot_hand)

In [21]:
#normalize function
numeric_columns_1 = ['player_ht', 'opponent_ht']
numeric_columns_2 = ['player_age','opponent_age']
numeric_columns_3 = ['player_rank', 'opponent_rank']
numeric_columns_4 = ["year"]

def normalize(og_df, numeric_columns):
    df = og_df.copy()
    for column in numeric_columns:
        # Z-score normalization
        df[f'{column}_zscore'] = (df[column] - df[column].mean()) / df[column].std()
        # Min-max scaling
        df[f'{column}_minmax'] = (df[column] - df[column].min()) / (df[column].max() - df[column].min())
        # Decimal scaling
        max_length = max(df[column].apply(lambda x: len(str(np.floor(x))) if x > 0 else 1))
        df[f'{column}_decimal'] = df[column] / (10**max_length)
    df.drop(columns=numeric_columns, inplace=True)
    return df

In [22]:
#no normalization (NN) has home
df_hand_int_encoded_home, df_hand_1hot_encoded_home, df_hand_int_encoded_bool, df_hand_1hot_encoded_bool = encode_pipeline_L1(df_home)

#normalize all
enc_int_N_Outlier_filled_dropped_nulls = normalize(df_hand_int_encoded_home,numeric_columns_1+numeric_columns_2+numeric_columns_3+numeric_columns_4)
enc_1hot_N_Outlier_filled_dropped_nulls = normalize(df_hand_1hot_encoded_home,numeric_columns_1+numeric_columns_2+numeric_columns_3+numeric_columns_4)

#normalize year
enc_int_Nyear_Outlier_filled_dropped_nulls_bool = normalize(df_hand_int_encoded_bool,numeric_columns_4)
enc_1hot_Nyear_Outlier_filled_dropped_nulls_bool = normalize(df_hand_1hot_encoded_bool,numeric_columns_4)

In [23]:
#df without outlier
df_hand_int_encoded_outlier, df_hand_1hot_encoded_outlier,df_hand_int_encoded_bool_outlier,df_hand_1hot_encoded_bool_outlier = encode_pipeline_L1(df_outlier2)

#normalize all
enc_int_N_noOutlier_filled_dropped_nulls = normalize(df_hand_int_encoded_outlier,numeric_columns_1+numeric_columns_2+numeric_columns_3+numeric_columns_4)
enc_1hot_N_noOutlier_filled_dropped_nulls = normalize(df_hand_1hot_encoded_outlier,numeric_columns_1+numeric_columns_2+numeric_columns_3+numeric_columns_4)

#normalize year
enc_int_Nyear_noOutlier_filled_dropped_nulls_bool = normalize(df_hand_int_encoded_bool_outlier,numeric_columns_4)
enc_1hot_Nyear_noOutlier_filled_dropped_nulls_bool = normalize(df_hand_1hot_encoded_bool_outlier,numeric_columns_4)

In [24]:
#df without nulls
df_no_nulls = df_drop_nulls.copy()

df_hand_int_encoded_nnull, df_hand_1hot_encoded_nnull,df_hand_int_encoded_bool_nnull,df_hand_1hot_encoded_bool_nnull = encode_pipeline_L1(df_no_nulls)

#normalize all
enc_int_N_Outlier_dropped_nulls = normalize(df_hand_int_encoded_nnull,numeric_columns_1+numeric_columns_2+numeric_columns_3+numeric_columns_4)
enc_1hot_N_Outlier_dropped_nulls = normalize(df_hand_1hot_encoded_nnull,numeric_columns_1+numeric_columns_2+numeric_columns_3+numeric_columns_4)

#normalize year
enc_int_Nyear_Outlier_dropped_nulls_bool = normalize(df_hand_int_encoded_bool_nnull,numeric_columns_4)
enc_1hot_Nyear_Outlier_dropped_nulls_bool = normalize(df_hand_1hot_encoded_bool_nnull,numeric_columns_4)

In [25]:
from sklearn.preprocessing import StandardScaler
from sklearn.svm import LinearSVC
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

#split df into training and testing
def split_df(df):
    working_df = df.copy()
    X = working_df.drop(['win', 'player', 'opponent'], axis=1)  # Exclude non-numeric or target columns
    y = working_df['win']
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)
    return X_train, X_test, y_train, y_test

#train Linear SVC model
def lin_svc_model(X_train, X_test, y_train, y_test):
    # Initialize the SVM classifier
    lsvm_model = LinearSVC(dual=True, max_iter=10000, random_state=1)
    lsvm_model.fit(X_train, y_train)
    y_pred_lsvm = lsvm_model.predict(X_test)
    print(classification_report(y_test, y_pred_lsvm))
    return y_test, y_pred_lsvm

#train SVC model
def svc_model(X_train, X_test, y_train, y_test):
    # Initialize the SVM classifier
    svm_model = SVC(kernel='linear', probability=True, random_state=1)
    svm_model.fit(X_train, y_train)
    y_pred_svm = svm_model.predict(X_test)
    print(classification_report(y_test, y_pred_svm))
    return y_test, y_pred_svm, svm_model, X_test

#train random forest model
def rfc_model(X_train, X_test, y_train, y_test):
    # Initialize the Random Forest classifier
    rf_model = RandomForestClassifier(n_estimators=100, random_state=1)
    rf_model.fit(X_train, y_train)
    y_pred_rf = rf_model.predict(X_test)
    print(classification_report(y_test, y_pred_rf))
    return y_test, y_pred_rf, rf_model, X_test

#train logistic regression model
def logReg_model(X_train, X_test, y_train, y_test):
    # Initialize the Logistic Regression classifier
    lr_model = LogisticRegression(random_state=1)
    lr_model.fit(X_train, y_train)
    y_pred_lr = lr_model.predict(X_test)
    print(classification_report(y_test, y_pred_lr))
    return y_test, y_pred_lr, lr_model, X_test

In [ ]:
#split df into training and testing
def split_df(df):
    working_df = df.copy()
    X = working_df.drop(['win', 'player', 'opponent'], axis=1)  # Exclude non-numeric or target columns
    y = working_df['win']
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)
    return X_train, X_test, y_train, y_test

#train random forest model
def rfc_model(X_train, X_test, y_train, y_test):
    # Initialize the Random Forest classifier
    rf_model = RandomForestClassifier(n_estimators=100, random_state=1)
    rf_model.fit(X_train, y_train)
    y_pred_rf = rf_model.predict(X_test)
    return y_test, y_pred_rf, rf_model, X_test

def calcMetrics(y_test, y_pred, model, X_test):
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    roc_auc = roc_auc_score(y_test, model.predict_proba(X_test)[:, 1])  # For binary classifications

    print(f"Accuracy: {accuracy}")
    print(f"Precision: {precision}")
    print(f"Recall: {recall}")
    print(f"F1 Score: {f1}")
    print(f"ROC-AUC Score: {roc_auc}")

def rfc_pipeline(og_df):
    print("Random Forest Classifier")
    df = og_df.copy()
    X_train, X_test, y_train, y_test = split_df(df)
    y_test, y_pred, model, X_test = rfc_model(X_train, X_test, y_train, y_test)
    calcMetrics(y_test, y_pred, model, X_test)

In [26]:
# Calculate metrics
def calcMetric(y_test, y_pred):
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)

    print(f"Accuracy: {accuracy}")
    print(f"Precision: {precision}")
    print(f"Recall: {recall}")
    print(f"F1 Score: {f1}")

def calcMetrics(y_test, y_pred, model, X_test):
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    roc_auc = roc_auc_score(y_test, model.predict_proba(X_test)[:, 1])  # For binary classifications

    print(f"Accuracy: {accuracy}")
    print(f"Precision: {precision}")
    print(f"Recall: {recall}")
    print(f"F1 Score: {f1}")
    print(f"ROC-AUC Score: {roc_auc}")

In [27]:
def lin_svc_pipeline(og_df):
    print("Linear Support Vector Classifier")
    df = og_df.copy()
    X_train, X_test, y_train, y_test = split_df(df)
    y_test, y_pred= lin_svc_model(X_train, X_test, y_train, y_test)
    calcMetric(y_test, y_pred)

def svc_pipeline(og_df):
    print("Support Vector Classifier")
    df = og_df.copy()
    X_train, X_test, y_train, y_test = split_df(df)
    y_test, y_pred, model, X_test = svc_model(X_train, X_test, y_train, y_test)
    calcMetrics(y_test, y_pred, model, X_test)

def rfc_pipeline(og_df):
    print("Random Forest Classifier")
    df = og_df.copy()
    X_train, X_test, y_train, y_test = split_df(df)
    y_test, y_pred, model, X_test = rfc_model(X_train, X_test, y_train, y_test)
    calcMetrics(y_test, y_pred, model, X_test)
    
def logReg_pipeline(og_df):
    print("Logistic regression Classifier")
    df = og_df.copy()
    X_train, X_test, y_train, y_test = split_df(df)
    y_test, y_pred, model, X_test = logReg_model(X_train, X_test, y_train, y_test)
    calcMetrics(y_test, y_pred, model, X_test)

In [37]:
# dataframes = [df_hand_int_encoded_NN,df_hand_1hot_encoded_NN,df_hand_int_N_all,df_hand_1hot_N_all]
# dataframe_names = ["hand_int_encoded_NN","hand_1hot_encoded_NN","hand_int_N_all","hand_1hot_N_all"]

dataframes = [enc_int_N_Outlier_filled_dropped_nulls,enc_1hot_N_Outlier_filled_dropped_nulls,enc_int_Nyear_Outlier_filled_dropped_nulls_bool,enc_1hot_Nyear_Outlier_filled_dropped_nulls_bool,
              enc_int_N_noOutlier_filled_dropped_nulls,enc_1hot_N_noOutlier_filled_dropped_nulls,enc_int_Nyear_noOutlier_filled_dropped_nulls_bool,enc_1hot_Nyear_noOutlier_filled_dropped_nulls_bool,
              enc_int_N_Outlier_dropped_nulls,enc_1hot_N_Outlier_dropped_nulls,enc_int_Nyear_Outlier_dropped_nulls_bool,enc_1hot_Nyear_Outlier_dropped_nulls_bool]
dataframe_names = ["enc_int_N_Outlier_filled_dropped_nulls","enc_1hot_N_Outlier_filled_dropped_nulls","enc_int_Nyear_Outlier_filled_dropped_nulls_bool","enc_1hot_Nyear_Outlier_filled_dropped_nulls_bool",
              "enc_int_N_noOutlier_filled_dropped_nulls","enc_1hot_N_noOutlier_filled_dropped_nulls","enc_int_Nyear_noOutlier_filled_dropped_nulls_bool","enc_1hot_Nyear_noOutlier_filled_dropped_nulls_bool",
              "enc_int_N_Outlier_dropped_nulls","enc_1hot_N_Outlier_dropped_nulls","enc_int_Nyear_Outlier_dropped_nulls_bool","enc_1hot_Nyear_Outlier_dropped_nulls_bool"]
enc_int_N_Outlier_filled_dropped_nulls.columns

Index(['Round', 'player', 'opponent', 'player_hand', 'opponent_hand', 'win',
       'is_player_home_country', 'is_opponent_home_country', 'month_sin',
       'month_cos', 'dayofweek_sin', 'dayofweek_cos', 'day_sin', 'day_cos',
       'player_ht_zscore', 'player_ht_minmax', 'player_ht_decimal',
       'opponent_ht_zscore', 'opponent_ht_minmax', 'opponent_ht_decimal',
       'player_age_zscore', 'player_age_minmax', 'player_age_decimal',
       'opponent_age_zscore', 'opponent_age_minmax', 'opponent_age_decimal',
       'player_rank_zscore', 'player_rank_minmax', 'player_rank_decimal',
       'opponent_rank_zscore', 'opponent_rank_minmax', 'opponent_rank_decimal',
       'year_zscore', 'year_minmax', 'year_decimal'],
      dtype='object')

In [29]:
raw,a,b,c = encode_pipeline_L1(raw_df)
# raw = raw.dropna()
rfc_pipeline(raw)

Random Forest Classifier
              precision    recall  f1-score   support

           0       0.78      0.77      0.77      4071
           1       0.76      0.77      0.77      3898

    accuracy                           0.77      7969
   macro avg       0.77      0.77      0.77      7969
weighted avg       0.77      0.77      0.77      7969

Accuracy: 0.7703601455640607
Precision: 0.7628368073207931
Recall: 0.7698819907644946
F1 Score: 0.7663432073544433
ROC-AUC Score: 0.8642538061264782


In [30]:
rawRaw = raw.copy()
rawRaw.drop(columns=["Unnamed: 0"], inplace=True)

rfc_pipeline(rawRaw)

Random Forest Classifier
              precision    recall  f1-score   support

           0       0.80      0.79      0.79      4071
           1       0.78      0.79      0.79      3898

    accuracy                           0.79      7969
   macro avg       0.79      0.79      0.79      7969
weighted avg       0.79      0.79      0.79      7969

Accuracy: 0.7894340569707617
Precision: 0.7834525025536262
Recall: 0.7870702924576706
F1 Score: 0.7852572306117226
ROC-AUC Score: 0.8808442034341945


In [31]:
rawRawRaw = rawRaw.copy()
rawRawRaw.drop(columns=["W_wt"], inplace=True)
rfc_pipeline(rawRawRaw)

Random Forest Classifier
              precision    recall  f1-score   support

           0       0.78      0.77      0.77      4071
           1       0.76      0.77      0.77      3898

    accuracy                           0.77      7969
   macro avg       0.77      0.77      0.77      7969
weighted avg       0.77      0.77      0.77      7969

Accuracy: 0.7707366043418246
Precision: 0.7626172964747654
Recall: 0.771421241662391
F1 Score: 0.7669940058665986
ROC-AUC Score: 0.8610885930707367


In [32]:
i = 0
for df in dataframes:
    print(" \n"+dataframe_names[i])
    lin_svc_pipeline(df)
    print(" \n"+dataframe_names[i])
    rfc_pipeline(df)
    print(" \n"+dataframe_names[i])
    logReg_pipeline(df)
    i+=1

 
enc_int_N_Outlier_filled_dropped_nulls
Linear Support Vector Classifier
              precision    recall  f1-score   support

           0       0.63      0.61      0.62      3583
           1       0.63      0.64      0.63      3584

    accuracy                           0.63      7167
   macro avg       0.63      0.63      0.63      7167
weighted avg       0.63      0.63      0.63      7167

Accuracy: 0.6288544718850286
Precision: 0.6250676773145641
Recall: 0.6442522321428571
F1 Score: 0.6345149766419346
 
enc_int_N_Outlier_filled_dropped_nulls
Random Forest Classifier
              precision    recall  f1-score   support

           0       0.69      0.69      0.69      3583
           1       0.69      0.69      0.69      3584

    accuracy                           0.69      7167
   macro avg       0.69      0.69      0.69      7167
weighted avg       0.69      0.69      0.69      7167

Accuracy: 0.6873168689828381
Precision: 0.6876222408493993
Recall: 0.6866629464285714
F1 Sc

c:\Users\erich\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\svm\_base.py:1237: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


              precision    recall  f1-score   support

           0       0.51      1.00      0.68      1511
           1       0.74      0.01      0.03      1449

    accuracy                           0.51      2960
   macro avg       0.63      0.50      0.35      2960
weighted avg       0.62      0.51      0.36      2960

Accuracy: 0.5148648648648648
Precision: 0.7407407407407407
Recall: 0.013802622498274672
F1 Score: 0.02710027100271003
 
enc_int_N_Outlier_dropped_nulls
Random Forest Classifier
              precision    recall  f1-score   support

           0       0.88      0.88      0.88      1511
           1       0.88      0.88      0.88      1449

    accuracy                           0.88      2960
   macro avg       0.88      0.88      0.88      2960
weighted avg       0.88      0.88      0.88      2960

Accuracy: 0.8807432432432433
Precision: 0.8774104683195593
Recall: 0.8792270531400966
F1 Score: 0.8783178214408824
ROC-AUC Score: 0.9543209470553873
 
enc_int_N_Outlier_

c:\Users\erich\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


              precision    recall  f1-score   support

           0       0.61      0.60      0.60      1511
           1       0.59      0.59      0.59      1449

    accuracy                           0.60      2960
   macro avg       0.60      0.60      0.60      2960
weighted avg       0.60      0.60      0.60      2960

Accuracy: 0.5966216216216216
Precision: 0.5871496924128503
Recall: 0.5928226363008972
F1 Score: 0.5899725274725275
ROC-AUC Score: 0.6314736332001029
 
enc_1hot_N_Outlier_dropped_nulls
Linear Support Vector Classifier


...

c:\Users\erich\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


              precision    recall  f1-score   support

           0       0.59      0.58      0.59      1511
           1       0.57      0.59      0.58      1449

    accuracy                           0.58      2960
   macro avg       0.58      0.58      0.58      2960
weighted avg       0.58      0.58      0.58      2960

Accuracy: 0.5827702702702703
Precision: 0.5721024258760108
Recall: 0.5859213250517599
F1 Score: 0.5789294237981589
ROC-AUC Score: 0.60033140909612
 
enc_1hot_Nyear_Outlier_dropped_nulls_bool
Linear Support Vector Classifier


c:\Users\erich\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\svm\_base.py:1237: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


              precision    recall  f1-score   support

           0       0.51      0.99      0.67      1511
           1       0.50      0.01      0.01      1449

    accuracy                           0.51      2960
   macro avg       0.51      0.50      0.34      2960
weighted avg       0.51      0.51      0.35      2960

Accuracy: 0.510472972972973
Precision: 0.5
Recall: 0.006211180124223602
F1 Score: 0.012269938650306749
 
enc_1hot_Nyear_Outlier_dropped_nulls_bool
Random Forest Classifier
              precision    recall  f1-score   support

           0       0.81      0.79      0.80      1511
           1       0.79      0.80      0.80      1449

    accuracy                           0.80      2960
   macro avg       0.80      0.80      0.80      2960
weighted avg       0.80      0.80      0.80      2960

Accuracy: 0.797972972972973
Precision: 0.7888662593346911
Recall: 0.8019323671497585
F1 Score: 0.7953456536618754
ROC-AUC Score: 0.879790667837743
 
enc_1hot_Nyear_Outlier_dr

c:\Users\erich\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [34]:
rawRaw = rawRaw.dropna()
lin_svc_pipeline(rawRaw)
# svc_pipeline(rawRaw)
rfc_pipeline(rawRaw)
logReg_pipeline(rawRaw)

Linear Support Vector Classifier


c:\Users\erich\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\svm\_base.py:1237: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


              precision    recall  f1-score   support

           0       0.49      1.00      0.66      1444
           1       0.86      0.02      0.03      1516

    accuracy                           0.49      2960
   macro avg       0.67      0.51      0.34      2960
weighted avg       0.68      0.49      0.34      2960

Accuracy: 0.4945945945945946
Precision: 0.8571428571428571
Recall: 0.0158311345646438
F1 Score: 0.031088082901554404
Random Forest Classifier
              precision    recall  f1-score   support

           0       0.89      0.92      0.91      1444
           1       0.92      0.89      0.91      1516

    accuracy                           0.91      2960
   macro avg       0.91      0.91      0.91      2960
weighted avg       0.91      0.91      0.91      2960

Accuracy: 0.9060810810810811
Precision: 0.9199457259158752
Recall: 0.8944591029023746
F1 Score: 0.9070234113712374
ROC-AUC Score: 0.9702487867182189
Logistic regression Classifier
              precision 

c:\Users\erich\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
